# IV. Продвинутые темы

In [2]:
from z3 import *

# 1. Выражения, сорта и декларации

Рассмотрим декларации:

x, y : Int<br>
b : Bool<br>
f : Int → Bool<br>
g : Bool × Int → Int<br>

Проверьте корректность и определите сорт следующих выражений:<br>
f(x)<br>
g(b, x)<br>
f(g(b, x))<br>
g(f(x), y)<br>
g(x, b)

In [ ]:
x, y = Ints('x y')
b = Bool('b')
f = Function('f', IntSort(), BoolSort())
g = Function('g', BoolSort(), IntSort(), IntSort())

print (x.sort())

print (f(x).sort())
print ((x + 1).sort())
print (g(b, x).sort())
print (f(g(b, x)).sort())
print (g(f(x), y).sort())
# print (g(x, b).sort())  # ошибка, выражение некорректно

Рассмотрим выражение:

E = f(g(b, x)) ∧ f(g(b, y)) ∧ (g(b, x) > g(b, y))

Упростим выражение через ввод новых переменных

In [11]:
t1, t2 = Ints('t1 t2')

E = And(f(g(b, x)), f(g(b, y)), g(b, x) > g(b, y))
print("E: ", E)

s = Solver()
s.add(E)
print(s.check())
print(s.model())

SE = substitute(E, (g(b, x), t1), (g(b, y), t2))
print("SE: ", SE)

s = Solver()
s.add(SE)
print(s.check())
print(s.model())


E:  And(f(g(b, x)), f(g(b, y)), g(b, x) > g(b, y))
sat
[x = 2,
 b = False,
 y = 3,
 g = [(False, 3) -> 0, else -> 1],
 f = [else -> True]]
SE:  And(f(t1), f(t2), t1 > t2)
sat
[t1 = 0, t2 = -1, f = [else -> True]]


# 2. Массивы


Заданы декларации:

a : Array[Int, Int]

i, j : Int

v : Int

Рассмотрим следующие выражения:

select(store(a, i, v), j)

store(store(a, i, v), i, v)

select(store(store(a, i, v), j, v), i)

Задание

Для каждого выражения:

Укажите его сорт.

Упростите выражение с точностью до логической эквивалентности, используя только аксиому массивов.


In [ ]:
A = Array("A", IntSort(), IntSort())
i, j, v = Ints("i j v")
x, y = Ints("x y")  # для аксиом

print((Select(Store(A, i, v), j)).sort())  # Int
print(Store(Store(A, i, v), i, v).sort())  # Array(Int, Int)
print(Select(Store(Store(A, i, v), j, v), i).sort())  # Int

# аксиомы массивов (Array Axioms):
# Аксиома 1: select(store(a, x, v), y) = v  если x == y
# Аксиома 2: select(store(a, x, v), y) = select(a, y)  если x != y

ax = [
    Implies(x == y, Select(Store(A, x, v), y) == v),
    Implies(x != y, Select(Store(A, x, v), y) == Select(A, y))
]

# Анализ выражения: Select(Store(Store(A, i, v), i, v), i)

stm = Select(Store(Store(A, i, v), i, v), i)

result = simplify(stm)
print(f"Упрощенное выражение: {result}")
print(f"Результат: {result == v}")

# Доказываем, что Select(Store(Store(A, i, v), i, v), i) == v
s = Solver()
s.add(ax)
s.add(Not(stm == v))
print(f"Доказательство (unsat означает, что выражение эквивалентно v): {s.check()}")



Заданы:

a, b : Array[Int, Int]

i, j : Int

И ограничения:

a1 = store(a, i, 1)
a2 = store(a1, j, 2)
b1 = store(b, j, 2)
b2 = store(b1, i, 1)

Являются ли массивы a2 и b2 равными?


In [ ]:
A = Array('A', IntSort(), IntSort())
B = Array('B', IntSort(), IntSort())
i, j = Ints('i j')

A1 = Store(A, i, 1)
A2 = Store(A1, j, 2)

B1 = Store(B, j, 2)
B2 = Store(B1, i, 1)

print(A2 == B2)
prove(Implies(i != j, A2 == B2))  # не равны, могут быть различия на других позициях, кроме i j

# Докажем, что на i и j позициях = равные элементы
prove(Implies(i != j, And(A2[i] == B2[i], A2[j] == B2[j])))

# 3. Кванторы

Заданы декларации:

a : Array[Int, Int]

f : Int → Int

Рассмотрите формулы:

∀i : Int. select(a, i) ≥ 0

∃i : Int. select(a, i) < 0

∀i : Int. ∃j : Int. select(a, j) = i

∃j : Int. ∀i : Int. select(a, j) = i

Задание

Укажите, выполнима ли формула (имеет ли модель).

Если формула выполнима:

опишите возможный вид модели;

укажите, где Z3 будет использовать скулемизацию.

Если формула невыполнима — укажите формальную причину.

In [ ]:
# Проверим каждую формулу

a = Array('a', IntSort(), IntSort())
f = Function('f', IntSort(), IntSort())
i, j = Ints('i j')

print("∀i : Int. select(a, i) ≥ 0")
# Формула 1: ∀i : Int. select(a, i) ≥ 0 (все элементы неотрицательны)
formula_1 = ForAll(i, Select(a, i) >= 0)
s = Solver()
s.add(formula_1)
result_1 = s.check()
print(f"Выполнимость: {result_1}")
if result_1 == sat:
    print("МОДЕЛЬ СУЩЕСТВУЕТ:")
    print(s.model())

print("∃i : Int. select(a, i) < 0")
# Формула 2: ∃i : Int. select(a, i) < 0 (существует отрицательный элемент)
formula_2 = Exists(i, Select(a, i) < 0)
s = Solver()
s.add(formula_2)
result_2 = s.check()
print(f"Выполнимость: {result_2}")
if result_2 == sat:
    print("МОДЕЛЬ СУЩЕСТВУЕТ:")
    print(s.model())

print("∀i : Int. ∃j : Int. select(a, j) = i")
# Формула 3: ∀i : Int. ∃j : Int. select(a, j) = i
# Это невыполнимо, потому что требует всех целых чисел в массиве
formula_3 = ForAll(i, Exists(j, Select(a, j) == i))
s = Solver()
s.add(Not(formula_3))
result_3 = s.check()
print(f"Выполнимость: {result_3}")  # от обратного - sat - значит не для всех i выполлняется условие

print("∃j : Int. ∀i : Int. select(a, j) = i")
# Формула 4: ∃j : Int. ∀i : Int. select(a, j) = i
# Это невыполнимо: одна ячейка не может быть одновременно разными значениями
formula_4 = Exists(j, ForAll(i, Select(a, j) == i))
s = Solver()
s.add(formula_4)
result_4 = s.check()
print(f"Выполнимость: {result_4}")


# 4. Оптимизация

Есть три вида ресурсов: x, y, z (вещественные, ≥ 0).

Ограничения:

x + y + z = 500

x ≤ 200

y ≤ 300

z ≤ 250

Стоимость:

x стоит:

30 за единицу, если x ≤ 100,

45 за единицу, если x > 100;

y стоит 25 за единицу;

z стоит:

40 за единицу, если z ≥ 50,

60 за единицу, если z < 50.

Требуется

Минимизировать суммарную стоимость.

Найти оптимальные значения x, y, z и минимальную цену.

In [ ]:
x, y, z, cost = Reals('x y z cost')

opt = Optimize()

opt.add(x + y + z == 500)
opt.add(x <= 200)
opt.add(y <= 300)
opt.add(z <= 250)

cost_x = If(x <= 100, x * 30, x * 45)
cost_y = y * 25
cost_z = If(z >= 50, z * 40, z * 60)

opt.add(cost == cost_x + cost_y + cost_z)
opt.minimize(cost)

print(opt.check())
print(opt.model())

Рассматривается прямоугольный контейнер с ограничениями:

a + b ≤ 100

2a + 3b ≤ 180

a > 0, b > 0

Объём контейнера задаётся нелинейной функцией:

V=a⋅b⋅(100−a−b)

Требуется

Максимизировать объем

Запустить решение и зафиксировать результат check().

Если результат unknown, вывести reason_unknown().

In [ ]:
a, b, v = Reals('a b v')

opt = Optimize()

opt.add(a + b <= 100)
opt.add(2 * a + 3 * b <= 180)
opt.add(And(a > 0, b > 0))

v = a * b * (100 - a - b)

opt.maximize(v)

# print(opt.check())
# print(opt.model())
# print(opt.reason_unknown()) 
# задача нерешаема Z3, он не может найти оптимум на бесконечном поле значений (Real)

In [ ]:
a, b, v = Ints('a b v')

opt = Optimize()

opt.add(a + b <= 100)
opt.add(2 * a + 3 * b <= 180)
opt.add(And(a > 0, b > 0))

v = a * b * (100 - a - b)

opt.maximize(v)

print(opt.check())
print(opt.model())
# на дискретном поле получается лучше

# 5. Множественные солверы


Даны целочисленные переменные x, y.

Общие ограничения:

x ≥ 0

y ≥ 0

x + y ≥ 10

Создаётся базовый солвер s_base, содержащий только общие ограничения.

Затем создаются два солвера:

s_even: добавляет условие, что x, y — чётные;

s_odd: добавляет условие, что x, y — нечётные.

Требуется

Проверить выполнимость обоих солверов.

Получить по одной модели из каждого солвера.

In [ ]:
x, y = Ints('x y')

s_base = Solver()
s_base.add(And(x > 0, y > 0, x + y >=10))

s_even = Solver()
s_even.add(And(x % 2 == 0, y % 2 == 0))
s_even.add(s_base.assertions())

print('Even solver')
print(s_even.check())
print(s_even.model())

s_odd = Solver()
s_odd.add(And(x % 2 != 0, y % 2 != 0))
s_odd.add(s_base.assertions())

print('\nOdd solver')
print(s_odd.check())
print(s_odd.model())

Даны вещественные переменные a, b.

Общее ядро условий:

a ≥ 0

b ≥ 0

a + b = 1

Рассматриваются две гипотезы:

Гипотеза H₁: a ≥ 0.7

Гипотеза H₂: b ≥ 0.7

Требуется

Создать солвер s_core с общими условиями.

Создать два независимых солвера s1, s2, скопировав в них условия из s_core.

В s1 добавить гипотезу H₁, в s2 — гипотезу H₂.

Проверить выполнимость каждой гипотезы.

Если солвер выполним, вывести модель.

In [ ]:
a, b = Reals('a b')

s_core = Solver()
s_core.add(And(a >= 0, b >= 0, a + b == 1))

s1 = Solver()
s1.add(a >= 0.7)
s1.add(s_core.assertions())

print('First hypothesis')
print(s1.check())
print(s1.model())

s2 = Solver()
s2.add(s_core.assertions())
s2.add(b >= 0.7)

print('Second hypothesis')
print(s2.check())
print(s2.model())